# Introduction

This tutorial introduces the use of multiple data sources, and shows how can one exchange information between them. The code given here is based on [this example](https://github.com/simphony/osp-core/blob/master/examples/multiple_wrappers_example.py) and builds on the [introduction on the CUDS API](./cuds_api.ipynb).

## Background
One of the main strengths of CUDS objects is their ability to share information between different underlying data sources interchangeably.
Using OSP-core's inner workings a data source can be represented as a CUDS object.
A data source can be in turn a database, a simulation engine, or any other software package, which is able to either generate or store information.

We refer to a CUDS object, which represents an underlying data source as a **wrapper**, as it wraps around the data source.
Wrappers use the CUDS API with the addition to some wrapper-specific methods, which will be discussed later on in this tutorial.

For a wrapper to be initialized, one needs some context for the underlying data source (e.g. location, credentials, etc.) for this we introduce an object called **session**.
Conceptually a session can be thought as an interoperability level, or in simple terms it handles the transition from the user-friendly CUDS API to the more task-specific syntax data sources tend to have.

## Let's get hands on
We start by importing the example namespace of OSP-core. If you haven't already, you should install the city ontology before:

In [1]:
!pico install city

INFO 2020-12-02 11:54:26,244 [osp.core.ontology.installation]: Will install the following namespaces: ['city']
INFO 2020-12-02 11:54:26,280 [osp.core.ontology.yml.yml_parser]: Parsing YAML ontology file /mnt/c/Users/dea/Documents/Projects/simphony/osp-core/osp/core/ontology/docs/city.ontology.yml
INFO 2020-12-02 11:54:26,331 [osp.core.ontology.yml.yml_parser]: You can now use `from osp.core.namespaces import city`.
INFO 2020-12-02 11:54:26,333 [osp.core.ontology.parser]: Loaded 403 ontology triples in total
INFO 2020-12-02 11:54:26,374 [osp.core.ontology.installation]: Installation successful


In [2]:
from osp.core.namespaces import city

The `pretty_print` function is part of our utilities module and is a convinient way to output the tree-like structure of a CUDS object.

In [3]:
from osp.core.utils import pretty_print

The `getpass` function is used to retrieve an input from an user. It prints a prompt, then reads input from the user until they press return.

In [4]:
from getpass import getpass

The next statements imports the second data source session we will use in this example, namely a database, or more precisely the ORM toolkit [SQLAlchemy](https://www.sqlalchemy.org/) which we will use in turn to connect to a PostgreSQL database.
It will throws an error if it cannot find the `SQLAlchemyWrapperSession` as it needs to be installed from a separate repository. Please refer [here](https://github.com/simphony/simphony-sqlalchemy) for installation instructions.

In [5]:
from osp.wrappers.sqlalchemy import SqlAlchemySession

Next we import a session object, which will provide the context to a simple simulation engine we developed for demonstrational purposes. It is already included in OSP-core.

In [6]:
from osp.wrappers.simdummy import SimDummySession

The following lines prompt the user to enter the information needed to create a connection to a running instance of a PostgreSQL database, where the data of our simulation will be stored. Please make sure to point to an existing and running instance of PostgreSQL. To install PostgreSQL on your machine, please refer to their [documentation](https://www.postgresql.org/download/).

The information is stored in a string `postgres_url`, which will later be passed to the `SQLAlchemyWrapperSession` object to initiate a connection with the data base.

In [7]:
print("Input data to connect to Postgres table!")
user = input("User: ")
pwd = getpass("Password: ")
db_name = input("Database name: ")
host = input("Host: ")
port = int(input("Port [5432]: ") or 5432)
postgres_url = 'postgres://%s:%s@%s:%s/%s' % (user, pwd, host, port, db_name)

Input data to connect to Postgres table!


In the next lines we create our small ontology-loving [**EMMO town** example](https://emmc.info/emmo-info/). Please pay a closer attention to the fourth line as there you can see the power of the `add` method and how with one statement one can add multiple `CITIZEN` CUDS objects simultaneously.

In [8]:
emmo_town = city.City(name='EMMO town')

emmo_town.add(city.Citizen(name='Emanuele Ghedini'), rel=city.hasInhabitant)
emmo_town.add(city.Citizen(name='Adham Hashibon'), rel=city.hasInhabitant)
emmo_town.add(city.Citizen(name='Jesper Friis'),
              city.Citizen(name='Gerhard Goldbeck'),
              city.Citizen(name='Georg Schmitz'),
              city.Citizen(name='Anne de Baas'),
              rel=city.hasInhabitant)

emmo_town.add(city.Neighborhood(name="Ontology"))
emmo_town.add(city.Neighborhood(name="User cases"))

<city.Neighborhood: 034f396c-8b0a-411e-873e-482935688386,  CoreSession: @0x7f4ede777950>

Next we grow the `Ontology` neighbourhood by adding some streets to it: namely *relationships* and *entities* (puns are intended).

In [9]:
ontology_uid = None
for neighbourhood in emmo_town.get(oclass=city.Neighborhood):
    if neighbourhood.name == "Ontology":
        ontology_uid = neighbourhood.uid
        neighbourhood.add(city.Street(name="Relationships"), rel=city.hasPart)
        neighbourhood.add(city.Street(name="Entities"), rel=city.hasPart)

Whenever you add a relationship to a CUDS object, OSP-core will always automatically add the `inverse` relationship. In our example case we can now retrieve from the "Ontology" neighbourhood `onto`, which town it belongs to.

In [10]:
onto = emmo_town.get(ontology_uid)
print(onto.get(rel=city.isPartOf)[0].name + ' is my city!')

EMMO town is my city!


Let's now store our small city persistently in the PostgreSQL database. Working with session objects is similar to the way one is used to work with files in Python. And when you ponder a bit about it, it makes kind of sense, as what one intends to do is store some information in a data storage.

Using Python's `with` statement the connection to the database will be maintained only within the scope of the `with` statement. After exiting its scope the connection will be closed automatically. We advise the use of the `with` statement as it automatically manages opening and closing of database connections.

First we open a connection to a PostgreSQL database through our `SqlAlchemyWrapperSession` and assign it to the session variable. Then we assign to the `wrapper` variable a `CITY_WRAPPER` and pass it the needed context with the `session=session`. From there on, we treat the `wrapper` variable as a normal CUDS object with the only difference that its internal state is managed by the `SqlAlchemyWrapperSession` behind the scenes. On the last line, we see the benefits of that by simply executing the `commit` command onto the `session` object. This will trigger a series of events, with the end result being that our **EMMO town** will be stored in the database.

In [12]:
with SqlAlchemySession(postgres_url) as session:
    wrapper = city.CityWrapper(session=session)
    wrapper.add(emmo_town)
    session.commit()

Next we show how one can use multiple data source wrappers simultaneously. First we open a connection to the PostgreSQL database through `SqlAlchemyWrapperSession` as shown above and assign it to the `db_session` variable. Then we retrieve the information we have previously stored to it using CUDS' `get` method and use the `pretty_print` function to output the information.

Then we open a connection to our demonstrational simulation engine through `DummySimWrapperSession` and assign it to the `sim_session` variable. As you can see opening the second simulation session is within the scope of the `SqlAlchemyWrapperSession`. 

In the scope of `DummySimSession`, we initialize a `CitySimWrapper`, pass it the context from the `sim_session` and assign it to the `sim_wrapper` variable. The `CitySimWrapper` consumes a city and a person. The magic happens in the following `run` method, which recognise that **Peter** is a person and it then transforms him into a citizen of the **EMMO town**. This new information is then stored in the `sim_emmo_town` automatically within the `run` method. We then output the information about Peter, who is now a citizen of **EMMO town**.

Finally we `update` our now outdated **EMMO town** in the database by using the `update` command. It checks for any inconsistencies between the **EMMO town** stored in the database, `db_emmo_town`, and the modified by our simulation engine town, `sim_emmo_town`. In our case it will find its new citizen **Peter** and it will add it to the **EMMO town** in the database instance of the town, `db_emmo_town`. This change is then in turn made persistent by calling the `commit` method, which will actually store the information the database.

In [12]:
with SqlAlchemySession(postgres_url) as db_session:
    db_wrapper = city.CityWrapper(session=db_session)
    db_emmo_town = db_wrapper.get(emmo_town.uid)
    print("The database contains the following information about the city:")
    pretty_print(db_emmo_town)

    # Working with a Simulation wrapper
    with SimDummySession() as sim_session:
        sim_wrapper = city.CitySimWrapper(numSteps=1,
                                          session=sim_session)
        new_inhabitant = city.Person(age=31, name="Peter")
        sim_emmo_town, _ = sim_wrapper.add(db_emmo_town, new_inhabitant)
        sim_session.run()
        print("The city has a new inhabitant:")
        pretty_print(sim_emmo_town.get(new_inhabitant.uid))

    # update database
    db_wrapper.update(sim_emmo_town)
    db_session.commit()


The database contains the following information about the city:
- Cuds object named <EMMO town>:
  uuid: 96a2f49c-8009-456a-ad69-1cb1dea7f128
  type: city.City
  superclasses: city.City, city.GeographicalPlace, city.PopulatedPlace, cuba.Entity
  values: coordinates: [0 0]
  description: 
    To Be Determined

   |_Relationship city.hasInhabitant:
   | -  city.Citizen cuds object named <Emanuele Ghedini>:
   | .  uuid: 08e05374-0187-4114-a114-085431aeebde
   | .  age: 25
   | -  city.Citizen cuds object named <Adham Hashibon>:
   | .  uuid: a084c4ba-c054-4afd-b194-b4e983743706
   | .  age: 25
   | -  city.Citizen cuds object named <Jesper Friis>:
   | .  uuid: 209926bc-6bcd-466c-a86a-807780e4789c
   | .  age: 25
   | -  city.Citizen cuds object named <Gerhard Goldbeck>:
   | .  uuid: 0a0a8488-7400-4f4e-9e44-dbf56782cd3c
   | .  age: 25
   | -  city.Citizen cuds object named <Georg Schmitz>:
   | .  uuid: cdfe72ec-fc15-4a14-a516-88b576bf0a27
   | .  age: 25
   | -  city.Citizen cuds obje

Finally to ensure our database has successfully interpreted our addition to **Emmo town**, we check its contents and print them using `pretty_print`.

In [13]:
with SqlAlchemySession(postgres_url) as db_session:
    db_wrapper = city.CityWrapper(session=db_session)
    db_emmo_town = db_wrapper.get(emmo_town.uid)
    print("The database contains the following information about the city:")
    pretty_print(db_emmo_town)

The database contains the following information about the city:
- Cuds object named <EMMO town>:
  uuid: 96a2f49c-8009-456a-ad69-1cb1dea7f128
  type: city.City
  superclasses: city.City, city.GeographicalPlace, city.PopulatedPlace, cuba.Entity
  values: coordinates: [0 0]
  description: 
    To Be Determined

   |_Relationship city.hasInhabitant:
   | -  city.Citizen cuds object named <Emanuele Ghedini>:
   | .  uuid: 08e05374-0187-4114-a114-085431aeebde
   | .  age: 25
   | -  city.Citizen cuds object named <Adham Hashibon>:
   | .  uuid: a084c4ba-c054-4afd-b194-b4e983743706
   | .  age: 25
   | -  city.Citizen cuds object named <Jesper Friis>:
   | .  uuid: 209926bc-6bcd-466c-a86a-807780e4789c
   | .  age: 25
   | -  city.Citizen cuds object named <Gerhard Goldbeck>:
   | .  uuid: 0a0a8488-7400-4f4e-9e44-dbf56782cd3c
   | .  age: 25
   | -  city.Citizen cuds object named <Georg Schmitz>:
   | .  uuid: cdfe72ec-fc15-4a14-a516-88b576bf0a27
   | .  age: 25
   | -  city.Citizen cuds obje